In [64]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [65]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [66]:
tf.keras.backend.clear_session()

In [67]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [68]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 13 #Referente aos 14 anos de input  

In [69]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - IDH Longevidade,São Paulo - IDH Renda,Contratação Comercial (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Consumo de Cimento (t)
0,2008,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,7770.929330,0.798050,0.833201,0.807995,5.483335e+08,...,0.813030,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094,13476.268000
1,2009,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8316.504885,0.801676,0.836979,0.805067,5.302764e+08,...,0.814375,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503,13304.301000
2,2010,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8835.509117,0.805351,0.840483,0.802285,5.046425e+08,...,0.815701,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891,14534.980000
3,2011,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,9192.769768,0.809082,0.843735,0.799634,4.854622e+08,...,0.817014,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640,14871.943000
4,2012,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.812878,0.846755,0.797095,4.867661e+08,...,0.818321,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,15159.683000
5,2012,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.818123,0.852119,0.792457,4.867661e+08,...,0.820160,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,15159.683000
6,2013,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,9197.980881,0.820299,0.851212,0.793328,5.225848e+08,...,0.820840,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810,15089.297000
7,2014,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,8264.309556,0.821991,0.849730,0.793855,5.676413e+08,...,0.821077,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,13714.511286
8,2015,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,7069.711323,0.830674,0.860278,0.784967,5.179392e+08,...,0.824371,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,11767.653429
9,2016,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,6155.235871,0.832462,0.858485,0.785237,4.524144e+08,...,0.824495,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,9811.084286


In [70]:
input_data = data.iloc[:-2, 1:-1]
input_data

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - IDH Longevidade,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,7770.929330,0.798050,0.833201,0.807995,5.483335e+08,2.546644e+09,...,11.760983,0.813030,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094
1,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8316.504885,0.801676,0.836979,0.805067,5.302764e+08,2.679945e+09,...,9.472585,0.814375,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503
2,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8835.509117,0.805351,0.840483,0.802285,5.046425e+08,2.891930e+09,...,9.332485,0.815701,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891
3,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,9192.769768,0.809082,0.843735,0.799634,4.854622e+08,3.091988e+09,...,10.985689,0.817014,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640
4,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.812878,0.846755,0.797095,4.867661e+08,3.189506e+09,...,8.159014,0.818321,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854
5,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.818123,0.852119,0.792457,4.867661e+08,3.189506e+09,...,8.159014,0.820160,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854
6,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,9197.980881,0.820299,0.851212,0.793328,5.225848e+08,3.093872e+09,...,7.913561,0.820840,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810
7,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,8264.309556,0.821991,0.849730,0.793855,5.676413e+08,2.779340e+09,...,10.350442,0.821077,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951
8,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,7069.711323,0.830674,0.860278,0.784967,5.179392e+08,2.561731e+09,...,12.454398,0.824371,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659
9,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,6155.235871,0.832462,0.858485,0.785237,4.524144e+08,2.516859e+09,...,13.111908,0.824495,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167


In [71]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15159.683000
5     15089.297000
6     13714.511286
7     11767.653429
8      9811.084286
9      9918.501000
10    10001.077000
11    10458.276000
12    11181.809000
13    12113.838000
14    12383.649000
15             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [35]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)  
train_input

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - IDH Longevidade,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.883514,0.969205,1.178569,-2.056298,0.017605,-1.722677,-1.964591,1.803820,1.261808,-1.069662,...,1.059506,-1.848156,-1.402939,1.684423,-2.178603,0.590854,0.206106,-1.567966,-2.533457,2.194279
1,-1.733876,1.179542,1.155704,-2.046301,0.391965,-1.425738,-1.481925,1.435740,0.862198,-0.629794,...,-0.002029,-1.480663,-1.383652,0.981576,-0.777850,0.689271,0.167129,-0.814062,-1.229626,0.481775
2,-1.129435,1.296738,1.207341,-0.629357,0.748092,-1.124856,-1.034157,1.086131,0.294908,0.069718,...,-0.067018,-1.118526,-1.407183,1.002668,-0.652210,0.675257,-1.186236,-1.158834,-0.462578,-0.751130
3,-0.589710,0.985755,1.081476,0.203205,0.993235,-0.819357,-0.618652,0.752819,-0.129560,0.729872,...,0.699865,-0.759981,-1.414943,0.853708,-1.122242,0.143034,-1.482516,-0.883599,0.207443,-0.985561
4,-0.110634,0.667847,0.625047,0.733374,1.159583,-0.508565,-0.232775,0.433632,-0.100704,1.051665,...,-0.611365,-0.403264,-0.084210,0.196862,-0.512804,-0.256798,-1.116513,-0.598812,0.363649,-0.940393
5,-0.110634,0.667847,0.625047,0.733374,1.159583,-0.079174,0.452629,-0.149397,-0.100704,1.051665,...,-0.611365,0.099139,-0.084210,0.196862,-0.512804,-0.256798,-1.116513,-0.598812,0.363649,-0.940393
6,0.307009,0.278859,0.136956,1.104567,0.996811,0.099061,0.336771,-0.039859,0.691978,0.736089,...,-0.725225,0.284845,0.436115,-1.042682,0.187973,-0.856667,-0.536879,-0.477318,0.172886,-1.049748
7,0.631304,-0.249194,-0.277194,1.039792,0.356150,0.237565,0.147370,0.026429,1.689098,-0.301808,...,0.405188,0.349483,0.816511,-0.899048,0.911153,-2.197724,0.510528,0.710999,0.638049,0.448687
8,0.847947,-0.726616,-0.792879,0.634411,-0.463552,0.948490,1.495170,-1.090847,0.589170,-1.019878,...,1.381165,1.248953,1.115031,-0.741504,1.165669,-1.504178,1.766329,1.397010,-0.001564,0.497445
9,0.988926,-1.123856,-1.085693,0.405247,-1.091042,1.094929,1.266085,-1.056937,-0.860922,-1.167948,...,1.686169,1.282982,1.120884,-0.673595,1.028259,-0.078680,1.467117,0.674641,-0.642098,-0.792739


In [36]:
reshaped_train_input = dfToInputRNN(train_input)

In [37]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.78709446, -1.1997478 , -1.18968099, -0.27402793,
         -1.26564967,  1.06236478,  0.13751197, -1.00173768,
         -1.06885905,  1.21006289, -1.06753995,  0.85796332,
          1.14981742, -0.61003589,  1.37555295, -0.63366661,
         -0.33633854,  1.46606085,  1.55205843, -1.89197197,
          1.15569582,  1.0168078 ,  0.60487545,  0.68514464,
          0.94068731,  1.55821276,  1.80323696,  1.35601575,
          1.46810839,  1.66070328,  1.16286603,  0.60284565,
         -1.89413413,  1.52102876,  1.17974371,  1.36205135,
          0.93842961, -1.7525565 , -1.78966564, -1.61127222,
         -0.55868553, -0.65956316,  0.46182883, -0.18821393,
          2.10693108,  1.35866401,  1.78598251,  1.89015757,
          1.46935856, -1.43651044,  1.35689633,  1.18202414,
          0.19068774,  1.48293979,  1.48377853,  1.53635924,
          1.66809735,  1.39996042,  1.37876519,  1.63347986,
          2.00963311, -0.82061447,  2.39668535, -1.22330659,
         -1.18680267,  0

In [47]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15159.683000
5     15089.297000
6     13714.511286
7     11767.653429
8      9811.084286
9      9918.501000
10    10001.077000
11    10458.276000
12    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [48]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

12    11181.809
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [49]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - IDH Longevidade,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
13,0.435975,-0.998122,-1.400185,-1.107089,-1.043214,0.942531,-0.444321,-1.070335,1.163167,2.664808,...,-2.313421,-0.091998,0.543096,-0.953224,-0.737723,0.262294,-0.271278,-0.470748,-1.062152,-1.522768


In [50]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.43597501, -0.9981219 , -1.40018519, -1.10708941,
         -1.04321391,  0.94253059, -0.44432079, -1.0703353 ,
          1.16316662,  2.6648078 , -0.28415403,  0.78842482,
          2.70456197, -0.13731248,  1.0620003 ,  0.14874796,
         -0.17746697,  1.94394596,  2.11975022, -2.30353113,
          1.50318835,  0.92591867,  0.21976702,  0.38034066,
          1.07012252,  2.38027465,  1.18721136,  0.68454647,
          0.96357222,  1.49963583,  1.10064062,  1.54938258,
         -1.601903  ,  1.82071202,  1.75021424,  1.09338904,
          0.63049382, -1.89207081, -2.1907463 , -2.64430562,
          0.34649211, -1.64566109,  0.29105292, -0.8938401 ,
          1.60067003,  1.08600407,  1.57190372,  1.48403903,
          1.96565138, -2.59491426,  2.94159734,  1.66925951,
          0.0371629 ,  1.62436238,  1.59912894,  1.82219669,
          1.96888178,  1.62404736,  1.59415473,  1.92575804,
          2.73371511,  1.06901998,  1.0388819 , -1.5223207 ,
         -2.31342098, -0

In [51]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

13    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [84]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', return_sequences=True, input_shape=(train_input.shape[1], 
                                                                                         train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=5000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [95]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, 0, seed)

        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [96]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4177572927, 2103839931, 4192856479, 3268094405, 1244380830, 2015197619, 2437738573, 3898904165, 1239356634, 1817220759, 3038174113, 2727702858, 784409853, 851059747, 534176938, 613725417, 4044590421, 327422652, 2595753833, 3125095947, 269457112, 2659637076, 255920110, 2601517571, 2941100276, 2912874804, 3685765851, 102051389, 3145071357, 2058963647, 3797492896, 3152054060, 1108987519, 2562687550, 3077706666, 4267708019, 1785069025, 3816012431, 618825655, 1916399018, 2823042686, 1856109099, 1665399691, 3219106510, 1338488926, 2037504357, 514087962, 2309228429, 1885957288, 1767065825, 140394596, 30734738, 1750515314, 2833043652, 4239749875, 347940946, 4273724619, 1703426026, 2304804009, 269826020, 2510118688, 2660899581, 1731174475, 3141297175, 2648918391, 714306561, 3253238425, 2713957419, 2881467953, 1145042949, 3671628600, 3028190607, 2782945163, 1941848226, 1692417989, 846877239, 907065501, 125216338, 1895625166, 2176916231, 448927856, 1301686885, 2363487704, 1394295595, 1273637980,

2023-09-02 11:47:48.702640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:47:48.705252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:47:48.708664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 135338.640625
winner_seed: 4177572927


Step: 1 ___________________________________________


2023-09-02 11:48:34.321231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:48:34.323749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:48:34.324419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 143255.109375


Step: 2 ___________________________________________


...

loss: 168029.359375


Step: 98 ___________________________________________


2023-09-02 13:03:22.023999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:03:22.025318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:03:22.026537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 240190.609375


Step: 99 ___________________________________________


2023-09-02 13:03:45.696583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:03:45.698549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:03:45.700144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 501533.6875


final_seed: 2304804009


In [97]:
trained_model, history = lstm_model(reshaped_train_input, 
                                    train_target, want_verbose=1, seed=winner_seed)

2023-09-02 13:04:06.407578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:04:06.410935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:04:06.412268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5000


2023-09-02 13:04:06.751581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:04:06.753740: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:04:06.756064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 4s 4s/step - loss: 165568480.0000
Epoch 2/5000
1/1 [==============================] - 0s 13ms/step - loss: 161923392.0000
Epoch 3/5000
1/1 [==============================] - 0s 12ms/step - loss: 17486942.0000
Epoch 4/5000
1/1 [==============================] - 0s 12ms/step - loss: 5086272.5000
Epoch 5/5000
1/1 [==============================] - 0s 11ms/step - loss: 4173222.2500
Epoch 6/5000
1/1 [==============================] - 0s 10ms/step - loss: 4700104.0000
Epoch 7/5000
1/1 [==============================] - 0s 10ms/step - loss: 4834253.0000
Epoch 8/5000
1/1 [==============================] - 0s 11ms/step - loss: 4352553.5000
Epoch 9/5000
1/1 [==============================] - 0s 11ms/step - loss: 5678144.0000
Epoch 10/5000
1/1 [==============================] - 0s 10ms/step - loss: 4563630.5000
Epoch 11/5000
1/1 [==============================] - 0s 11ms/step - loss: 4062834.7500
Epoch 12/5000
1/1 [==============================] - 0s 11ms/s

1/1 [==============================] - 0s 9ms/step - loss: 532230.8750
Epoch 97/5000
1/1 [==============================] - 0s 10ms/step - loss: 303647.4062
Epoch 98/5000
1/1 [==============================] - 0s 10ms/step - loss: 440675.3750
Epoch 99/5000
1/1 [==============================] - 0s 9ms/step - loss: 472394.2188
Epoch 100/5000
1/1 [==============================] - 0s 9ms/step - loss: 634762.9375
Epoch 101/5000
1/1 [==============================] - 0s 9ms/step - loss: 725040.7500
Epoch 102/5000
1/1 [==============================] - 0s 9ms/step - loss: 629301.8750
Epoch 103/5000
1/1 [==============================] - 0s 10ms/step - loss: 637566.0000
Epoch 104/5000
1/1 [==============================] - 0s 10ms/step - loss: 574335.0625
Epoch 105/5000
1/1 [==============================] - 0s 9ms/step - loss: 453488.8438
Epoch 106/5000
1/1 [==============================] - 0s 9ms/step - loss: 417616.8750
Epoch 107/5000
1/1 [==============================] - 0s 9ms/step - 

...

1/1 [==============================] - 0s 10ms/step - loss: 290246.3125
Epoch 1415/5000
1/1 [==============================] - 0s 10ms/step - loss: 208225.4219
Epoch 1416/5000
1/1 [==============================] - 0s 9ms/step - loss: 422777.3750
Epoch 1417/5000
1/1 [==============================] - 0s 10ms/step - loss: 417058.0000
Epoch 1418/5000
1/1 [==============================] - 0s 10ms/step - loss: 336470.0000
Epoch 1419/5000
1/1 [==============================] - 0s 10ms/step - loss: 408327.2188
Epoch 1420/5000
1/1 [==============================] - 0s 10ms/step - loss: 224447.5000
Epoch 1421/5000
1/1 [==============================] - 0s 10ms/step - loss: 359914.3125
Epoch 1422/5000
1/1 [==============================] - 0s 9ms/step - loss: 358755.0625
Epoch 1423/5000
1/1 [==============================] - 0s 10ms/step - loss: 332246.5312
Epoch 1424/5000
1/1 [==============================] - 0s 10ms/step - loss: 717539.6875
Epoch 1425/5000
1/1 [=============================

1/1 [==============================] - 0s 9ms/step - loss: 310399.5000
Epoch 1508/5000
1/1 [==============================] - 0s 9ms/step - loss: 298142.6562
Epoch 1509/5000
1/1 [==============================] - 0s 11ms/step - loss: 435865.8125
Epoch 1510/5000
1/1 [==============================] - 0s 10ms/step - loss: 336229.4688
Epoch 1511/5000
1/1 [==============================] - 0s 10ms/step - loss: 331216.4688
Epoch 1512/5000
1/1 [==============================] - 0s 9ms/step - loss: 144433.3594
Epoch 1513/5000
1/1 [==============================] - 0s 10ms/step - loss: 780983.6250
Epoch 1514/5000
1/1 [==============================] - 0s 10ms/step - loss: 638441.6250
Epoch 1515/5000
1/1 [==============================] - 0s 9ms/step - loss: 272331.5312
Epoch 1516/5000
1/1 [==============================] - 0s 10ms/step - loss: 452513.9375
Epoch 1517/5000
1/1 [==============================] - 0s 10ms/step - loss: 373304.6875
Epoch 1518/5000
1/1 [==============================]

1/1 [==============================] - 0s 11ms/step - loss: 228195.8438
Epoch 1601/5000
1/1 [==============================] - 0s 10ms/step - loss: 474589.1562
Epoch 1602/5000
1/1 [==============================] - 0s 10ms/step - loss: 240355.3281
Epoch 1603/5000
1/1 [==============================] - 0s 10ms/step - loss: 192908.2812
Epoch 1604/5000
1/1 [==============================] - 0s 10ms/step - loss: 412016.8750
Epoch 1605/5000
1/1 [==============================] - 0s 10ms/step - loss: 315622.1250
Epoch 1606/5000
1/1 [==============================] - 0s 10ms/step - loss: 546690.1875
Epoch 1607/5000
1/1 [==============================] - 0s 9ms/step - loss: 426792.3125
Epoch 1608/5000
1/1 [==============================] - 0s 10ms/step - loss: 298948.7812
Epoch 1609/5000
1/1 [==============================] - 0s 10ms/step - loss: 263653.2188
Epoch 1610/5000
1/1 [==============================] - 0s 10ms/step - loss: 323232.4688
Epoch 1611/5000
1/1 [============================

1/1 [==============================] - 0s 11ms/step - loss: 214658.3438
Epoch 1694/5000
1/1 [==============================] - 0s 10ms/step - loss: 368411.5000
Epoch 1695/5000
1/1 [==============================] - 0s 10ms/step - loss: 335191.1250
Epoch 1696/5000
1/1 [==============================] - 0s 10ms/step - loss: 312211.5625
Epoch 1697/5000
1/1 [==============================] - 0s 9ms/step - loss: 878584.3125
Epoch 1698/5000
1/1 [==============================] - 0s 10ms/step - loss: 359999.9688
Epoch 1699/5000
1/1 [==============================] - 0s 10ms/step - loss: 769972.3125
Epoch 1700/5000
1/1 [==============================] - 0s 9ms/step - loss: 333086.0625
Epoch 1701/5000
1/1 [==============================] - 0s 11ms/step - loss: 733006.6875
Epoch 1702/5000
1/1 [==============================] - 0s 10ms/step - loss: 706637.3750
Epoch 1703/5000
1/1 [==============================] - 0s 9ms/step - loss: 524502.8750
Epoch 1704/5000
1/1 [==============================

1/1 [==============================] - 0s 10ms/step - loss: 330463.5312
Epoch 1787/5000
1/1 [==============================] - 0s 9ms/step - loss: 406646.8750
Epoch 1788/5000
1/1 [==============================] - 0s 9ms/step - loss: 277132.2188
Epoch 1789/5000
1/1 [==============================] - 0s 10ms/step - loss: 169726.2344
Epoch 1790/5000
1/1 [==============================] - 0s 10ms/step - loss: 174131.8125
Epoch 1791/5000
1/1 [==============================] - 0s 10ms/step - loss: 107672.0938
Epoch 1792/5000
1/1 [==============================] - 0s 9ms/step - loss: 205132.0000
Epoch 1793/5000
1/1 [==============================] - 0s 11ms/step - loss: 200893.3281
Epoch 1794/5000
1/1 [==============================] - 0s 11ms/step - loss: 507488.2188
Epoch 1795/5000
1/1 [==============================] - 0s 10ms/step - loss: 281209.9375
Epoch 1796/5000
1/1 [==============================] - 0s 10ms/step - loss: 196722.4062
Epoch 1797/5000
1/1 [==============================

In [98]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

2023-09-02 13:04:30.486808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:04:30.488280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:04:30.489491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 755ms/step
[12113.838](test_target) - [11233.257](prediction) = 880.5811640624997


In [99]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.0726921694067974

In [100]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = lstm_model(dfToInputRNN(train_input), 
                                       dfToInputRNN(train_target), want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [101]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - IDH Longevidade,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.924099,-1.323126,-0.093200,-0.714550,-1.234806,-1.222044,-1.239766,1.235115,1.148205,-1.122043,...,1.412346,-1.227730,-0.489673,1.413722,-1.410322,-1.401911,0.736997,-1.258070,-1.315911,1.285551
1,-0.465059,0.229123,-1.175483,-0.699637,0.020377,-0.005384,0.030617,-0.021010,0.140893,-0.184474,...,-0.643250,0.005993,1.393821,-0.739164,0.614359,0.862152,0.676789,1.188450,0.209310,-0.132366
2,1.389158,1.094002,1.268682,1.414187,1.214429,1.227428,1.209149,-1.214105,-1.289098,1.306518,...,-0.769096,1.221737,-0.904148,-0.674558,0.795962,0.539759,-1.413786,0.069619,1.106602,-1.153185


train_target:


,São Paulo - Consumo de Cimento (t)
0,13304.301
1,14534.980
2,14871.943


test_input:


...

test_target:


,São Paulo - Consumo de Cimento (t)
13,12113.838


2023-09-02 13:09:01.968322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:09:01.970114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:09:01.971505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 636ms/step
Error: 880.5811640624997




[14947.1015625,
 15057.1953125,
 15255.275390625,
 15162.982421875,
 14074.322265625,
 11690.62890625,
 10684.927734375,
 10403.1123046875,
 10008.2041015625,
 10108.0263671875,
 11233.2568359375]

In [102]:
display(targets)
display(predictions)

[15159.683,
 15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14947.1015625,
 15057.1953125,
 15255.275390625,
 15162.982421875,
 14074.322265625,
 11690.62890625,
 10684.927734375,
 10403.1123046875,
 10008.2041015625,
 10108.0263671875,
 11233.2568359375]

In [103]:
mae = mean_absolute_error(predictions, targets)
mae

880.7845312499994

In [104]:
porcentage = mae/np.mean(targets)
porcentage

0.07210120979311888